In [2]:
import random
import numpy as np
from scipy.optimize import minimize
from functools import partial
import pandas as pd

# Define the boolean function you want to minimize
def boolean_function(x, l, days):
    # Your boolean function here
    # This example function returns the sum of the input vector multiplied by the input array
    cost = np.sum(x * l.cost.values)
    dates = l['date'].iloc[x]
    constraint = (dates - dates.shift()).dt.days
    return cost + abs(constraint.mean() - days) + constraint.std()


# Define the objective function for the simulated annealing algorithm
def objective_function(x, boolean_function, l, days):
    return boolean_function(x, l, days=days)

# Define the neighborhood function for the simulated annealing algorithm
def neighborhood_function(x):
    # Flip one randomly selected element of the input vector
    i = random.randint(0, len(x) - 1)
    x_new = x.copy()
    x_new[i] = 1 - x_new[i]
    return x_new

# Set the initial solution
x0 = np.random.randint(0, 2, size=10)
l = np.random.randint(0, 10, size=10)
l = pd.DataFrame({'date': pd.date_range('2020-01-01','2021-01-01', periods=10), 'cost': l})
days = 30

# Set the parameters for the simulated annealing algorithm
maxiter = 10000   # Maximum number of iterations
T0 = 1.0   # Initial temperature
Tf = 0.1   # Final temperature
alpha = 0.99   # Temperature reduction factor

# Define the bounds for the variables (in this case, all variables are binary)
bounds = [(0, 1)] * len(x0)

# Define the objective function for the minimize function
objective_func = partial(objective_function, boolean_function=boolean_function, l=l, days=days)

# Run the simulated annealing algorithm using the minimize function
result = minimize(objective_func, x0, method='L-BFGS-B', bounds=bounds, options={
    'maxiter': maxiter,
    'T0': T0,
    'Tf': Tf,
    'alpha': alpha,
    'neighborhood': neighborhood_function,
    'disp': True
})

# Print the best solution found
print(f"Best solution: {result.x}")
print(f"Objective function value: {result.fun}")

/var/folders/1c/h8t9_vd53gsgz_wm9kswm8wm0000gp/T/ipykernel_2079/3278302569.py:48: OptimizeWarning: Unknown solver options: T0, Tf, alpha, neighborhood
  result = minimize(objective_func, x0, method='L-BFGS-B', bounds=bounds, options={


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0        10 variables are exactly at the bounds

At iterate    0    f=  8.84083D+01    |proj g|=  1.00000D+00

At iterate    1    f=  6.18888D+01    |proj g|=  1.00000D+00

At iterate    2    f=  5.29998D+01    |proj g|=  1.00000D+00

At iterate    3    f=  5.29798D+01    |proj g|=  9.99933D-01
  ys=-5.246E-10  -gs= 1.997E-02 BFGS update SKIPPED

At iterate    4    f=  5.29599D+01    |proj g|=  9.99867D-01
  ys= 0.000E+00  -gs= 1.997E-02 BFGS update SKIPPED

At iterate    5    f=  5.29399D+01    |proj g|=  9.99800D-01
  ys= 0.000E+00  -gs= 1.997E-02 BFGS update SKIPPED

At iterate    6    f=  5.29199D+01    |proj g|=  9.99734D-01
  ys= 0.000E+00  -gs= 1.997E-02 BFGS update SKIPPED

At iterate    7    f=  5.29000D+01    |proj g|=  9.99667D-01
  ys= 0.000E+00  -gs= 1.997E-02 BFGS update SKIPPED

At iterate    8    f=  5.28800D+01    |proj g|=  9.99600D-01
  ys= 0.000E+00

In [1]:
import random
import numpy as np
import pandas as pd
from scipy.optimize import dual_annealing
from functools import partial

size = 10

# Define the boolean function you want to minimize
def boolean_function(x, l, days):
    cost = np.sum(x * l.cost.values)
    dates = l['date'].iloc[x]
    constraint = (dates - dates.shift()).dt.days
    return cost + abs(constraint.mean() - days) + constraint.std()

# Define the objective function for the simulated annealing algorithm
def objective_function(x, boolean_function, l, days):
    x_int = np.round(x).astype(int)
    return boolean_function(x_int, l, days)

# Define the bounds for the variables (in this case, all variables are binary)
bounds = [(0, 1)] * size

# Set the initial solution
x0 = np.random.randint(0, 2, size=size)

# Set the parameters for the simulated annealing algorithm
maxiter = 10000   # Maximum number of iterations
T0 = 1.0   # Initial temperature
Tf = 0.1   # Final temperature
alpha = 0.99   # Temperature reduction factor

# Create a sample dataframe
dates = pd.date_range('2023-01-01', periods=size, freq='M')
costs = np.random.randint(1, 100, size=size)
df = pd.DataFrame({'date': dates, 'cost': costs})

# Set the desired constraint value
days = 30

# Define the objective function for the minimize function
objective_func = partial(objective_function, boolean_function=boolean_function, l=df, days=days)

# Run the simulated annealing algorithm using the minimize function
result = dual_annealing(objective_func, bounds=bounds, maxiter=maxiter)

# Print the best solution found
print(f"Best solution: {np.round(result.x).astype(int)}")
print(f"Objective function value: {result.fun}")

Best solution: [0 0 0 0 0 0 0 0 0 0]
Objective function value: 30.0


## MILP

In [7]:
import numpy as np

#T 
n = 5
T = np.tri(n, n, -1, dtype=int).T
T

array([[0, 1, 1, 1, 1],
       [0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0]])

In [12]:
#x
x = np.random.randint(0, 2, size=n)
x

array([1, 0, 1, 1, 0])

In [14]:
#d
d = np.linspace(1, 10*n, n)
d

array([ 1.  , 13.25, 25.5 , 37.75, 50.  ])

In [17]:
#a
a = x@T - x 
a

array([-1,  1,  0,  1,  3])

In [20]:
#dp 
dp = d * x 
dp

array([ 1.  ,  0.  , 25.5 , 37.75,  0.  ])

In [22]:
#cost
N = 10
cost = d * x + N * x@(T - np.eye(n))

In [25]:
np.linalg.norm(cost)

63.33492322565805

In [133]:
n = 5
T = np.tri(n, n, -1, dtype=int).T
x = np.random.randint(0, 2, size=n)
d = np.linspace(1, 10*n, n)
N = 10
cost = d * x - N * (x@T * x)
print('d:', d)
print('x:', x)
print('d * x :', d * x)
print('N * (x@T * x):', N * (x@T * x))
print(f"cost: {np.linalg.norm(cost / np.sum(x))}")

d: [ 1.   13.25 25.5  37.75 50.  ]
x: [1 1 1 0 0]
d * x : [ 1.   13.25 25.5   0.    0.  ]
N * (x@T * x): [ 0 10 20  0  0]
cost: 2.155419526062927


In [155]:
n = 5
T = np.tri(n, n, -1, dtype=int).T
x = np.random.randint(0, 2, size=n)
d = np.linspace(1, 10*n, n)
N = 10
print('x:', x)
dp = d * x 
a = x@T * x
A = np.outer(dp, x) - np.outer(x, dp)
B = N * (np.outer(a, x) - np.outer(x, a))
print('A:')
print(A)
print('B:')
print(B)
print('A-B: ')
print(A-B)
print(np.linalg.norm(A-B, ord=1))

x: [0 0 1 0 1]
A:
[[  0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.  -24.5]
 [  0.    0.    0.    0.    0. ]
 [  0.    0.   24.5   0.    0. ]]
B:
[[  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0   0   0 -10]
 [  0   0   0   0   0]
 [  0   0  10   0   0]]
A-B: 
[[  0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.  -14.5]
 [  0.    0.    0.    0.    0. ]
 [  0.    0.   14.5   0.    0. ]]
14.5


In [ ]:
from scipy.optimize import milp



# Data

In [275]:
# Convert the lists to numpy arrays
N = 20
timestep = 10 
timestamps = np.linspace(0, N*5, N) + np.random.randint(-3, 3, size=N)
cloud_cover = np.random.randint(0, 10, size=N)
spatial_res = np.random.choice([10, 15, 30], size=N)

# Calculate the time difference between images in days
import pandas as pd
df = pd.DataFrame({'timestamps': timestamps, 'cloud_cover': cloud_cover, 'spatial_res': spatial_res})
df

,timestamps,cloud_cover,spatial_res
0,-3.000000,9,15
1,5.263158,3,30
2,12.526316,8,30
3,12.789474,0,30
4,23.052632,3,10
5,28.315789,8,10
6,29.578947,9,30
7,33.842105,8,15
8,44.105263,1,15
9,46.368421,3,15


## SCIPY MINIMIZE

In [306]:
from scipy.optimize import NonlinearConstraint


T = np.tri(N, N, -1, dtype=int).T
d = df['timestamps'].values

#constraint 1 : dense time series
def constraint(x):
    dp = d * x 
    a = x@T * x
    A = np.outer(dp, x) - np.outer(x, dp)
    B = (np.outer(a, x) - np.outer(x, a))
    return (A-timestep*B).ravel()

lb = -1 * np.ones(N**2)
ub = 5 * np.ones(N**2)
con1 = NonlinearConstraint(constraint, lb, ub)

#constraint 2 : integer 
def integer_constraint(x):
    return x - np.round(x)

cons = [con1, {'type': 'eq', 'fun': integer_constraint}]

bounds = [(0, 1)] * N

from scipy.optimize import minimize

def objective_function(x):
    selected_idx = np.where(x == 1)[0]
    cc = df['cloud_cover'].iloc[selected_idx].sum()
    sr = df['spatial_res'].iloc[selected_idx].sum()
    return cc + sr

x0 = np.random.randint(0, 2, size=N)
res = minimize(objective_function, x0, method='SLSQP', bounds=bounds, constraints=cons)

In [307]:
df['selected'] = res.x
df

,timestamps,cloud_cover,spatial_res,selected
0,-3.000000,9,15,0.0
1,5.263158,3,30,0.0
2,12.526316,8,30,0.0
3,12.789474,0,30,0.0
4,23.052632,3,10,0.0
5,28.315789,8,10,0.0
6,29.578947,9,30,0.0
7,33.842105,8,15,0.0
8,44.105263,1,15,0.0
9,46.368421,3,15,1.0


In [308]:
dfs = df[df['selected'] == 1]
dfs['diff'] = dfs['timestamps'].diff()
dfs

/var/folders/1c/h8t9_vd53gsgz_wm9kswm8wm0000gp/T/ipykernel_1687/503959543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['diff'] = dfs['timestamps'].diff()


,timestamps,cloud_cover,spatial_res,selected,diff
9,46.368421,3,15,1.0,NaN
10,54.631579,0,10,1.0,8.263158
11,56.894737,6,15,1.0,2.263158
13,67.421053,8,30,1.0,10.526316
15,76.947368,7,15,1.0,9.526316
19,100.000000,5,30,1.0,23.052632


In [264]:
b = np.random.randint(0,2,(4,4))
b 

array([[1, 0, 0, 1],
       [1, 1, 1, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 1]])

In [265]:
b[ b>0 ]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Test GA

In [238]:
from deap import creator, base , tools, algorithms
# Define the fitness function
def fitness(individual):
    """
    Calculates the fitness of an individual, which is the sum of the cloud cover and spatial resolution
    over the selected images, and a penalty for the deviation from the regular timestep.
    """
    selected_idx = np.where(individual == 1)[0]
    if len(selected_idx) == 0:
        return (9999,)
    time_diff = np.diff(timestamps[selected_idx]).astype("timedelta64[D]")
    def bias(x):
        if x > 0:
            return x * 2
        else :
            return x
    time_diff = np.array([bias(x) for x in time_diff])
    cc = np.sum(cloud_cover[selected_idx])
    sr = np.sum(spatial_res[selected_idx])
    return (cc + sr + np.power(np.abs(time_diff - timestep), 1).mean() + 40 * (1 - np.mean(individual)),)

# Define the genetic algorithm parameters
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 0, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run the genetic algorithm
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

/Users/arthurcalvi/Venv/Kayrros-default/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/arthurcalvi/Venv/Kayrros-default/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg 	min 
0  	100   	9999	9999
1  	49    	9999	9999
2  	64    	9999	9999
3  	63    	9999	9999
4  	59    	9999	9999
5  	56    	9999	9999
6  	52    	9999	9999
7  	57    	9999	9999
8  	62    	9999	9999
9  	66    	9999	9999
10 	62    	9999	9999
11 	56    	9999	9999
12 	65    	9999	9999
13 	63    	9999	9999
14 	58    	9999	9999
15 	61    	9999	9999
16 	65    	9999	9999
17 	52    	9999	9999
18 	54    	9999	9999
19 	55    	9999	9999
20 	63    	9999	9999
21 	65    	9999	9999
22 	63    	9999	9999
23 	61    	9999	9999
24 	63    	9999	9999
25 	60    	9999	9999
26 	58    	9999	9999
27 	51    	9999	9999
28 	53    	9999	9999
29 	49    	9999	9999
30 	53    	9999	9999
31 	68    	9999	9999
32 	67    	9999	9999
33 	49    	9999	9999
34 	57    	9999	9999
35 	62    	9999	9999
36 	57    	9999	9999
37 	51    	9999	9999
38 	50    	9999	9999
39 	66    	9999	9999
40 	54    	9999	9999
41 	58    	9999	9999
42 	57    	9999	9999
43 	63    	9999	9999
44 	65    	9999	9999
45 	60    	9999	9999
46 	62    	99

In [239]:
df['selectionned'] = hof[0]
df

,timestamps,cloud_cover,spatial_res,selectionned
0,4.000000,6,10,0
1,7.421053,3,30,1
2,14.842105,6,15,1
3,26.263158,9,30,0
4,30.684211,4,15,0
5,38.105263,9,15,0
6,51.526316,6,30,1
7,60.947368,7,30,1
8,71.368421,7,15,0
9,73.789474,0,10,0
